In [1]:
%pip install packaging
%pip install pandas dash


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.1
    Uninstalling Flask-3.1.1:
      Successfully uninstalled Flask-3.1.1


In [2]:
%pip install httpx==0.20 dash plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: h11
    Found existing installation: h11 0.16.0
    Uninstalling h11-0.16.0:
      Successfully uninstalled h11-0.16.0
  Attempting uninstall: anyio
    Found existing installation: anyio 4.9.0
    Uninstalling anyio-4.9.0:
      Successfully uninstalled anyio-4.9.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 1.0.9
    Uninstalling httpcore-1.0.9:
      Successfully uninstalled httpcore-1.0.9
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR:

In [3]:
import pandas as pd
import plotly.graph_objects as go
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px


In [4]:
spacex_df =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv',
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str,
                                   'Div2Airport': str, 'Div2TailNum': str})


In [5]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [6]:
# Contar los lanzamientos exitosos (class == 1) por sitio
total_exitos = spacex_df[spacex_df['class'] == 1]['Launch Site'].value_counts().reset_index()
total_exitos.columns = ['Launch Site', 'Total Éxitos']

print("Total de lanzamientos exitosos por sitio:")
print(total_exitos)

Total de lanzamientos exitosos por sitio:
    Launch Site  Total Éxitos
0    KSC LC-39A            10
1   CCAFS LC-40             7
2   VAFB SLC-4E             4
3  CCAFS SLC-40             3


In [7]:
# Agrupar por sitio y calcular total de lanzamientos y éxitos
lanzamientos = spacex_df.groupby('Launch Site')['class'].agg(['sum', 'count']).reset_index()
lanzamientos.columns = ['Launch Site', 'Éxitos', 'Total Lanzamientos']

# Calcular tasa de éxito
lanzamientos['Tasa de Éxito (%)'] = (lanzamientos['Éxitos'] / lanzamientos['Total Lanzamientos']) * 100

print("Tasa de éxito por sitio (%):")
print(lanzamientos.sort_values(by='Tasa de Éxito (%)', ascending=False))

Tasa de éxito por sitio (%):
    Launch Site  Éxitos  Total Lanzamientos  Tasa de Éxito (%)
2    KSC LC-39A      10                  13          76.923077
1  CCAFS SLC-40       3                   7          42.857143
3   VAFB SLC-4E       4                  10          40.000000
0   CCAFS LC-40       7                  26          26.923077


In [8]:
# 1. Definir los rangos de carga útil en intervalos de 1000 kg
bins = list(range(0, 11000, 1000))  # 0-1000, 1000-2000, ..., 9000-10000
labels = [f'{bins[i]}–{bins[i+1]} kg' for i in range(len(bins)-1)]

# 2. Crear columna con los rangos de carga útil
spacex_df['Payload Range'] = pd.cut(spacex_df['Payload Mass (kg)'], bins=bins, labels=labels, include_lowest=True)

# 3. Agrupar por rango y calcular tasa de éxito
payload_stats = spacex_df.groupby('Payload Range')['class'].agg(['sum', 'count']).reset_index()
payload_stats.columns = ['Payload Range', 'Éxitos', 'Total Lanzamientos']
payload_stats['Tasa de Éxito (%)'] = (payload_stats['Éxitos'] / payload_stats['Total Lanzamientos']) * 100

# 4. Mostrar resultados ordenados
print(payload_stats.sort_values(by='Tasa de Éxito (%)', ascending=False))

   Payload Range  Éxitos  Total Lanzamientos  Tasa de Éxito (%)
3   3000–4000 kg       8                  11          72.727273
9  9000–10000 kg       3                   5          60.000000
2   2000–3000 kg       5                  10          50.000000
5   5000–6000 kg       2                   5          40.000000
4   4000–5000 kg       3                   8          37.500000
1   1000–2000 kg       1                   3          33.333333
0      0–1000 kg       2                  10          20.000000
6   6000–7000 kg       0                   4           0.000000
7   7000–8000 kg       0                   0                NaN
8   8000–9000 kg       0                   0                NaN


<ipython-input-8-4928dd60afe7>:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  payload_stats = spacex_df.groupby('Payload Range')['class'].agg(['sum', 'count']).reset_index()


In [9]:
# Agrupar por la versión del booster
booster_stats = spacex_df.groupby('Booster Version')['class'].agg(['sum', 'count']).reset_index()

# Renombrar columnas
booster_stats.columns = ['Booster Version', 'Lanzamientos Exitosos', 'Total Lanzamientos']

# Calcular la tasa de éxito
booster_stats['Tasa de Éxito (%)'] = (booster_stats['Lanzamientos Exitosos'] / booster_stats['Total Lanzamientos']) * 100

# Ordenar de mayor a menor tasa de éxito
booster_stats_sorted = booster_stats.sort_values(by='Tasa de Éxito (%)', ascending=False)

# Mostrar resultados
print(booster_stats_sorted)

   Booster Version  Lanzamientos Exitosos  Total Lanzamientos  \
7    F9 B4 B1042.1                      1                   1   
6    F9 B4 B1041.1                      1                   1   
5    F9 B4 B1040.1                      1                   1   
4    F9 B4 B1039.1                      1                   1   
19     F9 FT B1019                      1                   1   
13  F9 FT  B1029.2                      1                   1   
14  F9 FT  B1031.2                      1                   1   
16  F9 FT  B1035.2                      1                   1   
8    F9 B4 B1043.1                      1                   1   
10   F9 B4 B1045.1                      1                   1   
11  F9 B5  B1046.1                      1                   1   
12  F9 FT  B1021.2                      1                   1   
29   F9 FT B1031.1                      1                   1   
30   F9 FT B1032.1                      1                   1   
26     F9 FT B1026       

In [10]:
# Create a dash application
app = dash.Dash(__name__)
##
min_payload = spacex_df['Payload Mass (kg)'].min()
max_payload = spacex_df['Payload Mass (kg)'].max()
##creo dropdown
# Obtener la lista de sitios únicos del DataFrame
launch_sites = spacex_df['Launch Site'].unique()

# Crear la lista de opciones para el menú desplegable
site_options = [{'label': 'All Sites', 'value': 'ALL'}] + \
               [{'label': site, 'value': site} for site in launch_sites]

# Crear el menú desplegable de selección de sitios de lanzamiento
dropdown_component = dcc.Dropdown(
    id='site-dropdown',  # ID para identificar este componente en los callbacks
    options=site_options,  # Lista de opciones con label y value
    value='ALL',  # Valor por defecto seleccionado
    placeholder='Select a Launch Site here',  # Texto de ayuda
    searchable=True  # Permite buscar escribiendo el nombre del sitio
)

# Build dash app layout
app.layout = html.Div(children=[
    html.H1(
        'SpaceX Launch Records Dashboard',
        style={'textAlign': 'center', 'color': '#503D36', 'font-size': 30}
    ),



    html.Br(),

       html.Div(dropdown_component, style={'textAlign': 'center'}),

    html.Br(),

    html.Div([
        html.Div(dcc.Graph(id='success-pie-chart')),
##      html.Div(dcc.Graph(id='weather-plot'))
    ], style={'display': 'flex'}),

    html.Div([
##        html.Div(dcc.Graph(id='nas-plot')),
##        html.Div(dcc.Graph(id='security-plot'))
    ], style={'display': 'flex'}),
    html.Br(),
##    html.Div(dcc.Graph(id='late-plot'), style={'width': '65%', 'margin': 'auto'})
        html.P("Payload range (Kg):"),
        dcc.RangeSlider(id='payload-slider',
                    min=0,
                    max=10000, step=1000,
                    marks={i: f'{i}' for i in range(0, 10001, 1000)},
                    value=[min_payload, max_payload]
),
# Control deslizante de payload
      html.P("Payload range (Kg):"),
      dcc.RangeSlider(id='payload-slider2',
                    min=0, max=10000, step=1000,
                    marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
                    value=[spacex_df['Payload Mass (kg)'].min(), spacex_df['Payload Mass (kg)'].max()]),

    html.Br(),
# Gráfico de dispersión
    dcc.Graph(id='success-payload-scatter-chart')

])

In [11]:
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        # Pie chart de todos los lanzamientos exitosos agrupados por sitio
        df_all = spacex_df[spacex_df['class'] == 1]
        fig = px.pie(df_all,
                     names='Launch Site',
                     title='Total de lanzamientos exitosos por sitio')
        return fig
    else:
        # Filtrar por sitio seleccionado
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        site_counts = filtered_df['class'].value_counts().reset_index()
        site_counts.columns = ['class', 'count']
        site_counts['class'] = site_counts['class'].replace({1: 'Success', 0: 'Failure'})

        fig = px.pie(site_counts,
                     values='count',
                     names='class',
                     title=f'Success vs Failure para {entered_site}')
        return fig
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [
        Input(component_id='site-dropdown', component_property='value'),
        Input(component_id='payload-slider', component_property='value')
    ]
)
def update_scatter_chart(selected_site, payload_range):
    # Filtrar por rango de payload
    low, high = payload_range
    mask = (spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)
    filtered_df = spacex_df[mask]

    # Lógica según sitio seleccionado
    if selected_site == 'ALL':
        fig = px.scatter(
            filtered_df,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category',
            title='Payload vs. Outcome for All Sites'
        )
    else:
        # Filtrar también por sitio de lanzamiento
        site_df = filtered_df[filtered_df['Launch Site'] == selected_site]
        fig = px.scatter(
            site_df,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category',
            title=f'Payload vs. Outcome for {selected_site}'
        )

    return fig

In [12]:
# Run the app
if __name__ == '__main__':
    app.run()

<IPython.core.display.Javascript object>